# 📊 Evaluación rigurosa de modelos RNN: Perplejidad y palabras fuera de vocabulario

Este notebook presenta una implementación estructurada para evaluar modelos LSTM en tareas de modelado de lenguaje, 
haciendo especial énfasis en métricas como la *Perplejidad* (PP) y el manejo de palabras fuera de vocabulario (*Out-Of-Vocabulary*, OOV).

## 🔧 BLOQUE 1: Setup y descarga de datos

- Instalación de dependencias necesarias (`torch`, `torchtext`, `nltk`, etc.).
- Descarga del dataset **WikiText-2**.
- Tokenización del corpus.
- Construcción de vocabulario con diferentes tamaños:
  - 10,000 palabras
  - 30,000 palabras
  - 50,000 palabras

## 🧠 BLOQUE 2: Definición del modelo

Se define un modelo LSTM con las siguientes características:

- Embeddings de dimensión 300.
- Dos capas LSTM con tamaño oculto de 512.
- Capa final densa con softmax para predecir la siguiente palabra.

**Componentes del modelo:**
- `Embedding layer`
- `LSTM layers (stacked)`
- `Linear output layer`

## 🏋️ BLOQUE 3: Entrenamiento

Entrenamos tres versiones del modelo, una por cada tamaño de vocabulario.

**Funciones incluidas:**
- Entrenamiento y validación por épocas.
- Registro de métricas:
  - *Perplejidad* sobre conjunto de validación.
  - Conteo y porcentaje de palabras OOV.

## 🧩 BLOQUE 4: Manejo de OOV

Se comparan tres estrategias de tratamiento para palabras fuera de vocabulario:

1. **Token `<UNK>`:** se reemplazan todas las palabras desconocidas por un token especial.
2. **Modelo char-level:** backoff a un modelo de caracteres para predecir embeddings.
3. **Similitud de Levenshtein:** reemplazo por la palabra más cercana en el vocabulario.

## 📈 BLOQUE 5: Evaluación

Métricas medidas:

- **Perplejidad** en el conjunto de prueba.
- **Porcentaje de OOV** en test.
- Comparación entre modelos con distintos vocabularios y estrategias OOV.

## 📊 BLOQUE 6: Análisis y visualización

Incluye:

- Gráficos de perplejidad vs. tamaño de vocabulario.
- Cobertura del vocabulario vs. porcentaje de OOV.
- Comparativa de estrategias de manejo de OOV.

## 📘 BLOQUE 7: Reporte final

Resumen de hallazgos:

- Impacto del tamaño del vocabulario sobre la perplejidad.
- Eficacia relativa de cada estrategia de OOV.
- Recomendaciones para aplicaciones en producción.

**Exportable a PDF o documento académico.**